# [Yargy](https://github.com/natasha/yargy)

Простая альтернатива [Tomita-парсеру](https://tech.yandex.ru/tomita/)

In [1]:
from yargy import (
    Parser,
    rule,
    and_, or_
)
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact, attribute
from yargy.predicates import (
    eq, gte, lte, length_eq,
    dictionary, normalized,
)

### Задача - научиться извлекать из документа ссылка на АПК РФ и т.п.

In [2]:
text = 'Апелляционная жалоба рассмотрена в порядке статей 123, 156 АПК РФ в отсутствие представителей лиц, участвующих в деле'
text_full = text.replace('АПК', 'Арбитражного Процессуального Кодекса')

In [3]:
def findall(gr, text):
    for match in Parser(gr).findall(text):
        print([_.value for _ in match.tokens]) 

### Тривиальное решение

In [4]:
GR = rule(
        'ст', 
        and_(gte(1), lte(1000))
     )

findall(GR, 'согласно ст 123')

['ст', '123']


### Добавим "`cт`" и "`статья`"

In [5]:
NUM = and_(gte(1), lte(1000))

GR = rule(
        or_(rule('статья'),
            rule('ст', eq('.').optional())
        ),
        and_(gte(1), lte(1000))
     )

findall(GR, 'согласно статьи 123')

### Добавим морфологию

In [6]:
GR = rule(
        or_(rule(normalized('статья')),
            rule('ст', eq('.').optional())
        ),
        and_(gte(1), lte(1000))
     )

findall(GR, 'согласно статьи 123')

['статьи', '123']


### Добавим "арбитражный процессуальный кодекс"

In [7]:
GR = rule(
        or_(rule(normalized('статья')),
            rule('ст', eq('.').optional())
        ),
        and_(gte(1), lte(1000)),
        morph_pipeline({
            'апк',
            'арбитражный процессуальный кодекс'
        })
     )

findall(GR, 'согласно статьи 123 арбитражно процессуального кодекса')

['статьи', '123', 'арбитражно', 'процессуального', 'кодекса']


### Несколько чисел

In [8]:
NUMBERS = rule(NUM, 
               rule(eq(','), NUM).repeatable().optional())

GR = rule(
        or_(rule(normalized('статья')),
            rule('ст', eq('.').optional())
        ),
        NUMBERS,
        morph_pipeline({
            'апк',
            'арбитражный процессуальный кодекс'
        })
     )

findall(GR, 'согласно статьям 123, 240 арбитражно процессуального кодекса')

['статьям', '123', ',', '240', 'арбитражно', 'процессуального', 'кодекса']


### Сохранение метаинформации 

In [9]:
Entry = fact(
      'Entry', 
      ['type', 'numbers']               
)

GR = rule(
        or_(rule(normalized('статья')),
            rule('ст', eq('.').optional())
        ),
        NUMBERS.interpretation(Entry.numbers),
        morph_pipeline({
            'апк',
            'арбитражный процессуальный кодекс'
        }).interpretation(Entry.type.const('АПК')),
    
     ).interpretation(Entry)

for match in Parser(GR).findall('согласно статьям 123, 240 арбитражно процессуального кодекса'):
    print(match.fact)

Entry(type='АПК', numbers='123, 240')


In [10]:
import natasha
from natasha import NamesExtractor, DatesExtractor

extractor = NamesExtractor()
matches = extractor('Конкурсный управляющий Макарова Л.В. в отзыве от 23.02.2016 возразил против доводов жалобы.')
for match in matches:
    print(match.span, match.fact)

[23, 36) Name(first='Л', middle='В', last='макаров', nick=None)
